In [3]:
import pandas as pd
import numpy as np
import glob
import warnings
import datetime

# Importación de recursos necesarios

In [2]:
# Titulos de los archivos a concatenar
#header = pd.read_csv('header.csv', sep=';', encoding='iso-8859-1')
#columns = header.columns.tolist()

columns = ['Sociedad','Numero de Documento','Ejercicio','Posicion','Cuenta de Mayor',\
           'Cuenta alternativa','Texto Posicion','Poliza (Clave Ref. 1)','Ramo (Clave Ref. 2)',\
           'Clave Referencia 3','Indicador D/H','Importe Moneda Local','Documento Compensacion',\
           'Fecha de Compensacion','Fecha de Contabilizacion','Periodo','Fecha de Documento',\
           'Fecha Fin Vigencia','Fecha Valor','Clase de Documento','Recibo (Referencia)',\
           'Texto Cabecera','Deudor','Acreedor','Nit','Nombre']


# Archivos CSV a concatenar
all_files = glob.glob('HARD_CLOSE_2100*.csv') ####### Cambiar ruta

# Concatenación de archivos

In [3]:
# Lista para almacenar los DataFrames de los archivos CSV
dataframes = []

# Cargar cada archivo CSV en un DataFrame y agregarlo a la lista
for file in all_files:
    warnings.filterwarnings('ignore')
    df = pd.read_csv(file, sep=';', header=None)
    dataframes.append(df)

# Concatenar los DataFrames en uno solo
maduracion = pd.concat(dataframes, ignore_index=True)
maduracion.columns = columns

# Formateo columnas

In [4]:
# Formateo de columnas tipo número
def format_value(value):
    if pd.notnull(value):
        return str(value).replace('.0', '')
    return value
columns_to_format = ['Recibo (Referencia)', 'Nit']
maduracion[columns_to_format] = maduracion[columns_to_format].applymap(format_value)

# Formateo de columnas tipo texto
maduracion['Nit'] = maduracion['Nit'].fillna('Vacío')
maduracion['Nombre'] = maduracion['Nombre'].fillna('Vacío')

# Formulas Maduración

In [5]:
# Fecha corte
fecha_corte = pd.to_datetime('2023/08/31') ####### Cambiar fecha

# Fecha cont
maduracion['Fecha cont'] = pd.to_datetime(maduracion['Fecha de Contabilizacion'], format='%Y%m%d').dt.strftime('%Y/%m/%d')

# Días entre la fecha de corte y la fecha contable ingresada
maduracion['Días Transcurridos'] = (fecha_corte - pd.to_datetime(maduracion['Fecha cont'])).dt.days.astype(int)

# 30 días
maduracion['30 días'] = maduracion['Importe Moneda Local'].where(maduracion['Días Transcurridos'] <= 30, '')

# 60 días
maduracion['60 días'] = maduracion['Importe Moneda Local'].where((maduracion['Días Transcurridos'] > 30) & \
                                                                 (maduracion['Días Transcurridos'] <= 60), '')

# 90 días
maduracion['90 días'] = maduracion['Importe Moneda Local'].where((maduracion['Días Transcurridos'] > 60) & \
                                                                 (maduracion['Días Transcurridos'] <= 90), '')

# 180 días
maduracion['180 días'] = maduracion['Importe Moneda Local'].where((maduracion['Días Transcurridos'] > 90) & \
                                                                 (maduracion['Días Transcurridos'] <= 180), '')

# 360 días
maduracion['360 días'] = maduracion['Importe Moneda Local'].where((maduracion['Días Transcurridos'] > 180) & \
                                                                 (maduracion['Días Transcurridos'] <= 360), '')

# >360 días
maduracion['>360 días'] = maduracion['Importe Moneda Local'].where(maduracion['Días Transcurridos'] > 360, '')

# Consolidación de información

In [6]:
# Agrupación de registros para consolidar información
groupby = maduracion.groupby(['Cuenta de Mayor', 'Recibo (Referencia)', 'Nit', 'Nombre', 'Fecha de Contabilizacion']) .agg({'30 días':'sum', '60 días':'sum', '90 días':'sum', '180 días':'sum', '360 días':'sum', '>360 días':'sum', 'Importe Moneda Local':'sum'}).reset_index()

# Registros de cuentas que la sumatoria es igual a cero
#cuentas_cero = maduracion[['Cuenta de Mayor', 'Nit', 'Nombre', 'Fecha de Contabilizacion', '30 días', '60 días', '90 días', '180 días', '360 días', '>360 días', 'Importe Moneda Local']].loc[maduracion['Clase de Documento'] == 'II']

# Concatenación de registros consolidados y registros con sumatoria cero
#final = pd.concat([groupby, cuentas_cero], ignore_index=True)

# Sumatoria para validar que el reporte es igual a al total de las cuentas solicitadas
total = groupby['Importe Moneda Local'].sum()

# Exportación

In [7]:
# Exportación de reporte en csv
groupby.to_csv('Febrero.csv', sep=';' ,encoding='utf-8-sig', index=False)
                ##### Cambiar nombre mes
# Exportación de valor total a txt
np.savetxt('Total.txt', [total], fmt='%d')

In [8]:
# Filtro
#### Opcional, no correr si no se requiere filtro
# filtro = maduracion[['Cuenta de Mayor', 'Nit', 'Nombre', 'Fecha de Contabilizacion', 'Días Transcurridos', 'Importe Moneda Local', '30 días', '60 días', '90 días', '180 días', '360 días', '>360 días']].loc[maduracion['Cuenta de Mayor'] == 2251100100]